In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml

#### Loading MNIST data set from openml

In [13]:
mnist = fetch_openml('mnist_784', version=1, cache=True)

In [14]:
X, y = mnist['data'], mnist['target']

In [18]:
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)
